In [1]:
import numpy as np

from qiskit import Aer
from qiskit.circuit import QuantumCircuit
from qiskit.quantum_info import Operator, Statevector
from qiskit.aqua.algorithms import IterativeAmplitudeEstimation as IQAE
from qiskit.aqua.algorithms import AmplitudeEstimation as QAE

from qiskit.aqua.components.uncertainty_problems.grover_operator import GroverOperator
from qiskit.aqua.components.uncertainty_problems.bit_oracle import BitOracle
from qiskit.aqua.utils import CircuitFactory
from qiskit.circuit.library import LinearPauliRotations
from qiskit.aqua.components.uncertainty_models import NormalDistribution

### Creating a bit-oracle 

This oracle is used to mark objective qubits.

In [2]:
s_f = BitOracle(3, [0, 2])  # 3 qubits, with objectives 0 and 2
s_f.draw()

┌───┐          ┌───┐     
state_0: ┤ X ├───────■──┤ X ├─────
         └───┘       │  └───┘     
state_1: ────────────┼────────────
         ┌───┐┌───┐┌─┴─┐┌───┐┌───┐
state_2: ┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├
         └───┘└───┘└───┘└───┘└───┘

### Grover operator

Constructs the Grover operator based on the oracle $\mathcal{S}_f$:
$$
\mathcal{Q} = \mathcal{A}\mathcal{S}_0\mathcal{A}^\dagger\mathcal{S}_f
$$
where $\mathcal{A}$ defaults to $H^{\otimes n}$, $\mathcal{S}_0 = \mathbb{I} - 2 |0\rangle\langle 0|$.

In [3]:
q = GroverOperator(s_f)

In [4]:
q.draw()

┌───┐          ┌───┐┌───┐┌───┐               ┌───┐┌───┐┌───┐     
state_0: ┤ X ├───────■──┤ X ├┤ H ├┤ X ├────────────■──┤ X ├┤ X ├┤ H ├─────
         ├───┤┌───┐  │  └───┘└───┘└───┘            │  ├───┤├───┤├───┤     
state_1: ┤ H ├┤ X ├──┼─────────────────────────────■──┤ X ├┤ X ├┤ H ├─────
         ├───┤├───┤┌─┴─┐┌───┐┌───┐┌───┐┌───┐┌───┐┌─┴─┐├───┤├───┤├───┤┌───┐
state_2: ┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ X ├┤ H ├
         └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘

In [5]:
GroverOperator(s_f, insert_barriers=True).decompose().draw()

┌─────────────┐                 ┌─────────────┐                ░ »
state_0: ┤ U3(pi,0,pi) ├──────────────■──┤ U3(pi,0,pi) ├────────────────░─»
         └─────────────┘              │  └─────────────┘                ░ »
state_1: ─────────────────────────────┼─────────────────────────────────░─»
         ┌─────────────┐┌──────────┐┌─┴─┐  ┌──────────┐ ┌─────────────┐ ░ »
state_2: ┤ U3(pi,0,pi) ├┤ U2(0,pi) ├┤ X ├──┤ U2(0,pi) ├─┤ U3(pi,0,pi) ├─░─»
         └─────────────┘└──────────┘└───┘  └──────────┘ └─────────────┘ ░ »
«         ┌──────────┐ ░ ┌─────────────┐                                       »
«state_0: ┤ U2(0,pi) ├─░─┤ U3(pi,0,pi) ├───────────────────────────────■───────»
«         ├──────────┤ ░ ├─────────────┤                               │       »
«state_1: ┤ U2(0,pi) ├─░─┤ U3(pi,0,pi) ├───────────────────■───────────┼───────»
«         ├──────────┤ ░ ├─────────────┤┌──────────┐┌───┐┌─┴─┐┌─────┐┌─┴─┐┌───┐»
«state_2: ┤ U2(0,pi) ├─░─┤ U3(pi,0,pi) ├┤ U2(0,pi) ├┤ H ├┤ X ├┤ TDG ├┤ X ├┤ T ├»
«         └──────────┘ ░ └─────────────┘└──────────┘└───┘└───┘└─────┘└───┘└───┘»
«                                ┌───┐             ┌─────────────┐»
«state_0: ──────────────■────■───┤ T ├──────■──────┤ U3(pi,0,pi) ├»
«               ┌───┐   │  ┌─┴─┐┌┴───┴┐   ┌─┴─┐    ├─────────────┤»
«state_1: ──■───┤ T ├───┼──┤ X ├┤ TDG ├───┤ X ├────┤ U3(pi,0,pi) ├»
«         ┌─┴─┐┌┴───┴┐┌─┴─┐├───┤└┬───┬┘┌──┴───┴───┐├─────────────┤»
«state_2: ┤ X ├┤ TDG ├┤ X ├┤ T ├─┤ H ├─┤ U2(0,pi) ├┤ U3(pi,0,pi) ├»
«         └───┘└─────┘└───┘└───┘ └───┘ └──────────┘└─────────────┘»
«         ┌─────────────┐ ░ ┌──────────┐
«state_0: ┤ U3(pi,0,pi) ├─░─┤ U2(0,pi) ├
«         ├─────────────┤ ░ ├──────────┤
«state_1: ┤ U3(pi,0,pi) ├─░─┤ U2(0,pi) ├
«         ├─────────────┤ ░ ├──────────┤
«state_2: ┤ U3(pi,0,pi) ├─░─┤ U2(0,pi) ├
«         └─────────────┘ ░ └──────────┘

### Bernoulli example

In [6]:
p = 0.2
angle = np.arcsin(np.sqrt(p))

a_operator = QuantumCircuit(1)
a_operator.ry(2 * angle, 0)
a_operator.value_to_estimation = lambda x: x  # TODO remove that this is necessary

q_operator = QuantumCircuit(1)
q_operator.ry(4 * angle, 0)

In [7]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [8]:
backend = Aer.get_backend('statevector_simulator')
ae = QAE(2, state_in=a_operator, grover_operator=q_operator)
result = ae.run(backend)
print(result['estimation'], result['mle'])

0.5 0.20000000018626451


In [9]:
ae.construct_circuit().decompose().draw()

┌──────────┐                                                ┌───┐
eval_0: ───┤ U2(0,pi) ├──────────■──────────────────────────────■──────┤ H ├
           ├──────────┤          │                        ┌───┐ │-pi/2 └───┘
eval_1: ───┤ U2(0,pi) ├──────────┼───────────────■────────┤ H ├─■───────────
        ┌──┴──────────┴──┐┌──────┴───────┐┌──────┴───────┐└───┘             
   q_0: ┤ R(0.9273,pi/2) ├┤ circuit54**1 ├┤ circuit54**2 ├──────────────────
        └────────────────┘└──────────────┘└──────────────┘

Using the default $\mathcal{Q}$ operator:

In [10]:
flag_last_qubit = BitOracle(1, [0])  # 0th qubit is objective qubit
q_default = GroverOperator(flag_last_qubit, a_operator)
print(q_default.draw())

         ┌───┐┌───┐┌───┐┌─────────────┐┌───┐┌───┐┌───┐┌───┐┌────────────┐
state_0: ┤ X ├┤ Z ├┤ X ├┤ RY(-0.9273) ├┤ X ├┤ Z ├┤ X ├┤ X ├┤ RY(0.9273) ├
         └───┘└───┘└───┘└─────────────┘└───┘└───┘└───┘└───┘└────────────┘


In [11]:
ae = QAE(2, a_operator, q_default)
result = ae.run(backend)
print(result['estimation'], result['mle'])

0.0 0.019677546808452455


### Idle qubits 

Idle qubits are ignored in the creation $\mathcal{S}_0$ reflection in the Grover operator.

In [12]:
p = 0.2
angle = np.arcsin(np.sqrt(p))

a_with_idle = QuantumCircuit(2)
a_with_idle.cry(2 * angle, 0, 1)
a_with_idle.value_to_estimation = lambda x: x

print(a_with_idle.draw())

                   
q_0: ──────■───────
     ┌─────┴──────┐
q_1: ┤ RY(0.9273) ├
     └────────────┘


We see that qubit 0 is not included in the $\mathcal{S}_0$ reflection:

In [13]:
s_f = BitOracle(2, [1])  # ignore first qubit
q_with_idle = GroverOperator(s_f, a_with_idle, idle_qubits=[0])
print(q_with_idle.decompose().draw())

                                                                              »
state_0: ─────────────────────────────────────────────────────────────────────»
         ┌─────────────┐┌──────────┐┌─────────────┐┌──────────┐┌─────────────┐»
state_1: ┤ U3(pi,0,pi) ├┤ U2(0,pi) ├┤ U3(pi,0,pi) ├┤ U2(0,pi) ├┤ U3(pi,0,pi) ├»
         └─────────────┘└──────────┘└─────────────┘└──────────┘└─────────────┘»
«                                                                         »
«state_0: ──────────────────────■───────────────────────■─────────────────»
«         ┌──────────────────┐┌─┴─┐┌─────────────────┐┌─┴─┐┌─────────────┐»
«state_1: ┤ U3(-0.46365,0,0) ├┤ X ├┤ U3(0.46365,0,0) ├┤ X ├┤ U3(pi,0,pi) ├»
«         └──────────────────┘└───┘└─────────────────┘└───┘└─────────────┘»
«                                                                              »
«state_0: ─────────────────────────────────────────────────────────────────────»
«         ┌──────────┐┌─────────────┐┌──────────┐┌────────

In [14]:
# initial_state = QuantumCircuit(2)
# initial_state.x(0)
# ae = QAE(2, a_factory=a_with_idle, q_factory=q_with_idle, initial_state=initial_state)
# print(ae.construct_circuit().draw())

In [15]:
# result = ae.run(backend)
# print(result['estimation'], result['mle'])

Let's turn the `X` gate off and then the rotation in the $\mathcal{A}$ operator does not act.

In [16]:
# initial_state = QuantumCircuit(2)
# ae = QAE(2, a_factory=a_with_idle, q_factory=q_with_idle, initial_state=initial_state)
# result = ae.run(backend)
# print(result['estimation'], result['mle'])

### Expectation value of a quadratic function

In [17]:
class ContinuousQuadratic(QuantumCircuit):
    def __init__(self, x_dist, y, rescaling_factor=0.1, name='A'):
        num_qubits = x_dist.num_target_qubits + 1
        super().__init__(num_qubits, name=name)
        self.c = rescaling_factor
        self.x_dist = x_dist
        self.y = y
        self.ub = x_dist.high
                
        indices = list(range(num_qubits - 1))
        self.i_x = indices[:self.x_dist.num_target_qubits]
        self.i_f = num_qubits - 1
        
        self._build()
        
    def _build(self):
        q = self.qubits
        beta_x = self.c / (self.x_dist.num_values - 1)
        gamma_y = -self.c * self.y / self.ub

        lin_x = LinearPauliRotations(self.x_dist.num_target_qubits, offset=2 * gamma_y, slope=2 * beta_x)

        q_x = [q[i] for i in self.i_x]
        q_f = q[self.i_f]
        self.x_dist.build(self, q_x)
        
        self.compose(lin_x, qubits=q_x[:] + [q_f], inplace=True)
        
    def value_to_estimation(self, value):
        return value * (self.ub / self.c) ** 2

In [18]:
X = NormalDistribution(2, mu=1, sigma=1, low=0, high=2)
a_X = ContinuousQuadratic(X, y=1)
bad_state_reflection = BitOracle(a_X.num_qubits, [a_X.i_f])
q_X = GroverOperator(bad_state_reflection, a_X, insert_barriers=False, mcx='v-chain')

In [19]:
BitOracle(6, [0, 1, 2, 3, 4, 5], mcx='v-chain').draw()

┌───┐           ┌───┐     
  state_0: ┤ X ├───────■───┤ X ├─────
           ├───┤       │   ├───┤     
  state_1: ┤ X ├───────■───┤ X ├─────
           ├───┤       │   ├───┤     
  state_2: ┤ X ├───────■───┤ X ├─────
           ├───┤       │   ├───┤     
  state_3: ┤ X ├───────■───┤ X ├─────
           ├───┤       │   ├───┤     
  state_4: ┤ X ├───────■───┤ X ├─────
           ├───┤┌───┐┌─┴──┐├───┤┌───┐
  state_5: ┤ X ├┤ H ├┤0   ├┤ H ├┤ X ├
           └───┘└───┘│    │└───┘└───┘
ancilla_0: ──────────┤1   ├──────────
                     │  X │          
ancilla_1: ──────────┤2   ├──────────
                     │    │          
ancilla_2: ──────────┤3   ├──────────
                     └────┘

#### Running canonical QAE

In [20]:
def post(x):
    return a_X.value_to_estimation(x)

In [21]:
ae = QAE(3, state_in=a_X, grover_operator=q_X, objective_qubits=[a_X.i_f], post_processing=post)
result = ae.run(backend)
print(result['estimation'], result['mle'])

0.0 8.651370895929562


In [22]:
print(ae.construct_circuit())

             ┌───┐                                                   »
eval_0: ─────┤ H ├───────────────────────────────────────────────────»
             ├───┤                                                   »
eval_1: ─────┤ H ├───────────────────────────────────────────────────»
             ├───┤                                                   »
eval_2: ─────┤ H ├───────────────────────────────────────────────────»
        ┌────┴───┴─────┐┌───┐┌─────────────────┐┌───┐                »
   q_0: ┤ U3(pi/2,0,0) ├┤ X ├┤ U3(0.22042,0,0) ├┤ X ├───────■────────»
        ├──────────────┤└─┬─┘└─────────────────┘└─┬─┘       │        »
   q_1: ┤ U3(pi/2,0,0) ├──■───────────────────────■─────────┼────────»
        └─┬──────────┬─┘                             ┌──────┴───────┐»
   q_2: ──┤ RY(-0.1) ├───────────────────────────────┤ RY(0.066667) ├»
          └──────────┘                               └──────────────┘»
«                       ┌─────────┐                      ┌──────┐
«eval_0: ──

In [23]:
# post(result['mle'])

#### Iterative QAE

In [24]:
ae = IQAE(0.01, 0.01, state_in=a_X, grover_operator=q_X, objective_qubits=[a_X.i_f], post_processing=post)
result = ae.run(backend)
print(result['estimation'])

0.4580526785717565


### Quadratic expecation value with qubit-encoded offset parameter

In [25]:
class DiscreteQuadratic(QuantumCircuit):
    def __init__(self, x_dist, k, y_max, rescaling_factor=0.1, name='A'):
        num_qubits = x_dist.num_target_qubits + k + 1
        super().__init__(num_qubits, name=name)
        self.c = rescaling_factor
        self.x_dist = x_dist
        self.k = k
        self.y_max = y_max
                
        indices = list(range(num_qubits - 1))
        self.i_x = indices[:self.x_dist.num_target_qubits]
        self.i_y = indices[self.x_dist.num_target_qubits:]
        self.i_f = num_qubits - 1
        
        self._build()
        
    def _build(self):
        beta_x = self.c / (self.x_dist.num_values - 1) * self.x_dist.high
        beta_y = -self.c / (2**self.k - 1) * self.y_max
        
        lin_x = LinearPauliRotations(self.x_dist.num_target_qubits, offset=0, slope=2 * beta_x)
        lin_y = LinearPauliRotations(self.k, offset=0, slope=2 * beta_y)

        q = self.qubits
        q_x = [q[i] for i in self.i_x]
        q_y = [q[i] for i in self.i_y]
        q_f = q[self.i_f]
        self.x_dist.build(self, q_x)
        
        self.compose(lin_x, qubits=q_x[:] + [q_f], inplace=True)
        self.compose(lin_y, qubits=q_y[:] + [q_f], inplace=True)   
        
    def value_to_estimation(self, value):
        return value / self.c ** 2

In [26]:
a_XY = DiscreteQuadratic(X, k=2, y_max=3)
init_y = QuantumCircuit(5)
init_y.x(2)
# X.build(init_y, init_y.qubits[:2])
s_f = BitOracle(5, [4])  # ignore first qubit
q_XY = GroverOperator(s_f, a_XY, idle_qubits=[2, 3])

ae = QAE(3, a_factory=a_XY, q_factory=q_XY, initial_state=init_y)

TypeError: __init__() got an unexpected keyword argument 'initial_state'

In [ ]:
result = ae.run(backend)
print(result['estimation'], result['mle'])

In [ ]:
ae = IQAE(0.01, 0.01, a_factory=a_XY, q_factory=q_XY, initial_state=init_y, i_objective=4)
result = ae.run(backend)
print(result['estimation'])

### Bernoulli with ancillas 

In [ ]:
from qiskit.circuit import AncillaRegister, QuantumRegister

p = 0.2
angle = np.arcsin(np.sqrt(p))

qr, ar = QuantumRegister(1), AncillaRegister(1)

a_operator = QuantumCircuit(qr)#, ar)
a_operator.ry(2 * angle, qr[0])
a_operator.value_to_estimation = lambda x: x  # TODO remove that this is necessary

In [ ]:
a_operator.draw()

In [ ]:
a_operator.num_ancillas

In [ ]:
num_state_qubits = a_operator.num_qubits - a_operator.num_ancillas
flag_last_qubit = BitOracle(num_state_qubits, [num_state_qubits - 1])  # 0th qubit is objective qubit
q_default = GroverOperator(flag_last_qubit, a_operator)
print(q_default.draw())

In [ ]:
backend = Aer.get_backend('qasm_simulator')
ae = IQAE(0.01, 0.01, a_factory=a_operator, q_factory=q_default)
result = ae.run(backend)
print(result['estimation'])#, result['mle'])

In [ ]:
ae.construct_circuit(2).draw()

In [ ]:
ae.construct_circuit().decompose().draw()

In [ ]:
class BernoulliAFactory(CircuitFactory):
    r"""Circuit Factory representing the operator A in a Bernoulli problem.

    Given a probability $p$, the operator A prepares the state $\sqrt{1 - p}|0> + \sqrt{p}|1>$.
    """

    def __init__(self, probability=0.5):
        #
        super().__init__(1)
        self._probability = probability
        self.i_state = 0
        self._theta_p = 2 * np.arcsin(np.sqrt(probability))

    def build(self, qc, q, q_ancillas=None, params=None):
        # A is a rotation of angle theta_p around the Y-axis
        qc.ry(self._theta_p, q[self.i_state])

    def value_to_estimation(self, value):
        return value

In [ ]:
a_factory = BernoulliAFactory(0.2)

In [ ]:
backend = Aer.get_backend('qasm_simulator')
ae = IQAE(0.01, 0.01, a_factory=a_factory)
result = ae.run(backend)
print(result['estimation'])#, result['mle'])

In [ ]:
ae.construct_circuit().decompose().draw()

In [ ]:
from qiskit.aqua.algorithms import VQE
from qiskit.aqua.operators import *
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import RealAmplitudes

op = (X ^ X) + (Z ^ Z) + (Y ^ Y)
ansatz = RealAmplitudes(2)
vqe = VQE(op, ansatz, expectation=MatrixExpectation())
exp = vqe.construct_circuit(ParameterVector('p', ansatz.num_parameters))
print(exp)

In [ ]:
print(exp)

In [ ]:
exp.oplist[0].to_circuit()